In [1]:
USER_KEY = "cc3948f0cd364a9f684a1fe78b9229ba"      #  Input your User key
DATA_KEY = "a71eaf04-802f-40be-93c2-5bee2548f4db"  #  Fixed Data Api key

print("USER_KEY:", USER_KEY if USER_KEY else "Not found")
print("DATA_KEY:", DATA_KEY if DATA_KEY else "Not found")

USER_KEY: cc3948f0cd364a9f684a1fe78b9229ba
DATA_KEY: a71eaf04-802f-40be-93c2-5bee2548f4db


In [2]:
import requests

root_url = f'https://aifapbt.fin.cloud.ainode.ai/'

headers = {"API-KEY": USER_KEY}
requests.get(root_url, headers=headers).json()

{'message': 'API Server is Running.',
 'status': 200,
 'User status': 'active',
 'Expires At': '2025-09-17T00:00:00',
 'Level': 1}

In [3]:
import os
strategy_name = "simple_reverting"


# Upload or Update strategy file
endpoint = 'upload/strategy'
url = root_url + endpoint

base_path = '/Users/seongmin/Desktop/test2/futures'
file_path = os.path.join(base_path, strategy_name, strategy_name + ".py")

params = {'tradeType':'futures', 'overwrite': True} # If overwrite is True, it will modify the existing file.

with open(file_path, "rb") as f:
    files = {"file": f}
    response = requests.post(url, headers=headers, params=params, files=files)

print("📂 Upload Response:", response.json())

📂 Upload Response: {'message': "'simple_reverting' Strategy Updated Successfully!"}


In [4]:
# Check strategy file upload
endpoint = 'upload/check/strategy'
url = root_url + endpoint

params = {"tradeType": "futures", "strategy_name": strategy_name}

response = requests.get(url, headers=headers, params=params)
print(response.json()['content'])

# strategy.py
import pandas as pd
import numpy as np

def strategy(df: pd.DataFrame, config_dict: dict) -> dict:
    """
    Implements a mean-reversion trading strategy based on Z-scores derived from
    rolling mean and standard deviation, conceptually inspired by the Ornstein-Uhlenbeck process.

    Args:
        df (pd.DataFrame): Price time-series data.
                           Index: datetime
                           Columns: symbols (e.g., BTCUSDT, ETHUSDT, …)
        config_dict (dict): Strategy settings dictionary.
                            Expected structure:
                            {
                                "strategy_config": {
                                    "long_window": int,
                                    "std_window": int,
                                    "entry_zscore": float,
                                    "exit_zscore": float,
                                    "max_weight_per_symbol": float
                                }
      

In [5]:
# Upload or Update strategy-config file
endpoint = 'upload/strategy-config'
url = root_url + endpoint

strategy_config_name = strategy_name + "_config"
base_path = '/Users/seongmin/Desktop/test2/futures'
file_path = os.path.join(base_path, strategy_name, strategy_config_name + ".py")

params = {"tradeType": "futures", "overwrite": True} # If overwrite is True, it will modify the existing file.

with open(file_path, "rb") as f:
    files = {"file": f}
    response = requests.post(url, headers=headers, params=params, files=files)

print("📂 Upload Response:", response.json())

📂 Upload Response: {'message': "'simple_reverting' Strategy-Config Updated Successfully!"}


In [6]:
# Check strategy-config file upload
endpoint = 'upload/check/strategy-config'
url = root_url + endpoint

params = {"tradeType": "futures", "strategy_name": strategy_name}

response = requests.get(url, headers=headers, params=params)
print(response.json()['content'])

# strategy_config.py
strategy_config = {
    "long_window": 60,  # Lookback period for the long-term rolling mean (e.g., 60 minutes for 1 hour mean)
    "std_window": 20,   # Lookback period for the rolling standard deviation used in Z-score calculation
    "entry_zscore": 1.5, # Z-score threshold to enter a position (e.g., 1.5 standard deviations from the mean)
    "exit_zscore": 0.5,  # Z-score threshold to exit a position (e.g., 0.5 standard deviations from the mean)
    "max_weight_per_symbol": 0.1 # Maximum absolute weight allocated to a single symbol before overall normalization
}


In [8]:
# Upload or Update system-config file
endpoint = "upload/system-config"
url = root_url + endpoint

system_config_name = "futures_config"
base_path = '/Users/seongmin/Desktop/test2/futures'
file_path = os.path.join(base_path, system_config_name + ".py")

params = {"tradeType": "futures", "overwrite": True} # If overwrite is True, it will modify the existing file.

with open(file_path, "rb") as f:
    files = {"file": f}
    response = requests.post(url, headers=headers, params=params, files=files)

print("📂 Upload Response:", response.json())

📂 Upload Response: {'message': "'Futures' System-Config Updated Successfully!"}


In [9]:
# Check system-config file upload
endpoint = "upload/check/system-config"
url = root_url + endpoint

params = {"tradeType": "futures"}

response = requests.get(url, headers=headers, params=params)
print(response.json()['content'])

# ==========================
# Required System Settings
# ==========================
system_config = {
    "data_apikey": "a71eaf04-802f-40be-93c2-5bee2548f4db",
    "tz_str": "Asia/Seoul", 
    "timeframe": "1min",
    "orderType": "market",
    "base_symbol": "BTCUSDT",
    "productType": "usdt-futures",
    "posMode": "hedge_mode",
    "holdSide": "long",                   
    "marginMode": "crossed",
    "marginCoin": "usdt",
    "symbols": ['BTCUSDT', 'ETHUSDT', 'LTCUSDT', 'TRXUSDT', 'DOTUSDT', 'DOGEUSDT', 'XRPUSDT', 'BCHUSDT', 'ADAUSDT'],
    "leverage": 5,
    "trading_hours": 6,
    "total_allocation": 0.4, 
}


# ==========================
# Rebalancing Trade Parameters
# ==========================
rebalancing_config = {
    "rebalancing_interval_hours": 1, # Rebalancing cycle (hours)
}



In [10]:
# run-system
endpoint = 'command/run-system'
url = root_url + endpoint

data = {
    "tradeType": "futures",
    "strategy_name": strategy_name,
    "trade_mode": "rebalancing",
    "trade_env": "live"
}

response = requests.post(url, headers=headers, json=data)

try:
    session_id = response.json()['session_id']
    print('session_id :',response.json()['session_id'])
    print('dashboard :',response.json()['dashboard_url'])
except:
    print('Error :',response.json()['message']['message'])

    session_id = response.json()['message']['session_id']
    print('session_id id :',session_id)

    print('dashboard :',response.json()['message']['dashboard_url'])

session_id : 0e037fbc
dashboard : https://aifapbt.fin.cloud.ainode.ai/dashboard/live_futures_trading_dashbaord.html


In [11]:
# Terminate process
# You must execute terminate to force liquidation of the current process.
endpoint = 'command/terminate'
url = root_url + endpoint

params = {"session_id": f"{session_id}", "tradeType": 'futures'}

response = requests.get(url, headers=headers, params=params)
print(response.json())

{'message': 'Session 0e037fbc ended'}
